# [Keras] 利用Keras建構LSTM模型
--以Stock Prediction 為例 1
### Authored by PJ Wang
### Apr 13, 2018
#### https://medium.com/@daniel820710/%E5%88%A9%E7%94%A8keras%E5%BB%BA%E6%A7%8Blstm%E6%A8%A1%E5%9E%8B-%E4%BB%A5stock-prediction-%E7%82%BA%E4%BE%8B-1-67456e0a0b


In [6]:
## import

import pandas as pd
import numpy as np
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
## load data
## https://finance.yahoo.com/quote/SPY/history?period1=728236800&period2=1523462400&interval=1d&filter=history&frequency=1d
## SPY dataset: Yahoo SPDR S&P 500 ETF (SPY)

def readTrain():
    dir_path = '../data' 
    file_path = dir_path + '/spy.csv'
    train = pd.read_csv(file_path)
    return train

In [39]:
## Augment Features

def augFeatures(train):
  train["Date"] = pd.to_datetime(train["Date"])
  train["year"] = train["Date"].dt.year
  train["month"] = train["Date"].dt.month
  train["date"] = train["Date"].dt.day
  train["day"] = train["Date"].dt.dayofweek
  return train

In [40]:
## Normalization

def normalize(train):
  train = train.drop(["Date"], axis=1)
  train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
  return train_norm

In [41]:
## Build Training Data

def buildTrain(train, pastDay=30, futureDay=5):
  X_train, Y_train = [], []
  for i in range(train.shape[0]-futureDay-pastDay):
    X_train.append(np.array(train.iloc[i:i+pastDay]))
    Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["Adj Close"]))
  return np.array(X_train), np.array(Y_train)

In [42]:
## disorderize data

def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

In [43]:
## Training data & Validation data

def splitData(X,Y,rate):
  X_train = X[int(X.shape[0]*rate):]
  Y_train = Y[int(Y.shape[0]*rate):]
  X_val = X[:int(X.shape[0]*rate)]
  Y_val = Y[:int(Y.shape[0]*rate)]
  return X_train, Y_train, X_val, Y_val

In [67]:
## main

# read SPY.csv
train = readTrain()

# Augment the features (year, month, date, day)
train_Aug = augFeatures(train)

# Normalization
train_norm = normalize(train_Aug)

# print(train[0:2])
# print(train_Aug[0:2])
# print(train_norm[0:2])


# Visualization
# print(train_norm.size)
# print(train_norm.shape)
# print(train_norm[0:2])
# print(train.iloc[0:30])
# print(np.array(train.iloc[0:30]))

# build Data, use last 30 days to predict next 5 days
X_train, Y_train = buildTrain(train_norm, 30, 5)

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, Y_train)

# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)
# X_trian: (5710, 30, 10)
# Y_train: (5710, 5, 1)
# X_val: (634, 30, 10)
# Y_val: (634, 5, 1)



        Date      Open      High       Low    Close  Adj Close   Volume  year  \
0 1993-01-29  43.96875  43.96875  43.75000  43.9375  26.299288  1003200  1993   
1 1993-02-01  43.96875  44.25000  43.96875  44.2500  26.486324   480500  1993   

   month  date  day  
0      1    29    4  
1      2     1    0  
        Date      Open      High       Low    Close  Adj Close   Volume  year  \
0 1993-01-29  43.96875  43.96875  43.75000  43.9375  26.299288  1003200  1993   
1 1993-02-01  43.96875  44.25000  43.96875  44.2500  26.486324   480500  1993   

   month  date  day  
0      1    29    4  
1      2     1    0  
       Open      High       Low     Close  Adj Close    Volume      year  \
0 -0.342052 -0.344423 -0.340762 -0.341338  -0.301050 -0.095759 -0.486789   
1 -0.342052 -0.343266 -0.339857 -0.340053  -0.300297 -0.096359 -0.486789   

      month      date       day  
0 -0.501146  0.442720  0.494918  
1 -0.410237 -0.490614 -0.505082  


In [46]:
## build 1-1 model

def buildOneToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
  # output shape: (1, 1)
  model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [47]:
## run 1-1 model

train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 1)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]

model = buildOneToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

d:\program\python\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
d:\program\python\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(1, 10))`
  """


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 10)             840       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5710 samples, validate on 634 samples
Epoch 1/1000
5710/5710 [==============================] - 1s 211us/step - loss: 0.0720 - val_loss: 0.0527
Epoch 2/1000
5710/5710 [==============================] - 0s 10us/step - loss: 0.0415 - val_loss: 0.0278
Epoch 3/1000
5710/5710 [==============================] - 0s 9us/step - loss: 0.0190 - val_loss: 0.0102
Epoch 4/1000
5710/5710 [==============================] - 0s 8us/step - loss: 0.0056 - val_loss: 0.0022
Epoch 5/1000
5710/5710 [============================

5710/5710 [==============================] - 0s 9us/step - loss: 4.0397e-05 - val_loss: 4.3068e-05
Epoch 69/1000
5710/5710 [==============================] - 0s 9us/step - loss: 4.0186e-05 - val_loss: 4.2151e-05
Epoch 70/1000
5710/5710 [==============================] - 0s 9us/step - loss: 3.9901e-05 - val_loss: 4.1465e-05
Epoch 71/1000
5710/5710 [==============================] - 0s 9us/step - loss: 3.9318e-05 - val_loss: 4.2272e-05
Epoch 72/1000
5710/5710 [==============================] - 0s 8us/step - loss: 3.9302e-05 - val_loss: 4.0940e-05
Epoch 73/1000
5710/5710 [==============================] - 0s 9us/step - loss: 3.8793e-05 - val_loss: 4.0711e-05
Epoch 74/1000
5710/5710 [==============================] - 0s 9us/step - loss: 3.8155e-05 - val_loss: 4.0290e-05
Epoch 75/1000
5710/5710 [==============================] - 0s 9us/step - loss: 3.7977e-05 - val_loss: 3.9750e-05
Epoch 76/1000
5710/5710 [==============================] - 0s 9us/step - loss: 3.8143e-05 - val_loss: 3.9882e-

5710/5710 [==============================] - 0s 8us/step - loss: 2.4606e-05 - val_loss: 2.4105e-05
Epoch 141/1000
5710/5710 [==============================] - 0s 8us/step - loss: 2.4804e-05 - val_loss: 2.3937e-05
Epoch 142/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.4658e-05 - val_loss: 2.4422e-05
Epoch 143/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.4237e-05 - val_loss: 2.3900e-05
Epoch 144/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.4050e-05 - val_loss: 2.3295e-05
Epoch 145/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.4148e-05 - val_loss: 2.3424e-05
Epoch 146/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.4126e-05 - val_loss: 2.3558e-05
Epoch 147/1000
5710/5710 [==============================] - 0s 8us/step - loss: 2.4392e-05 - val_loss: 2.2811e-05
Epoch 148/1000
5710/5710 [==============================] - 0s 7us/step - loss: 2.4590e-05 - val_loss: 

In [48]:
## build n-1 model

def buildManyToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
  # output shape: (1, 1)
  model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [49]:
## run n-1 model

train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 30, 1)
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

model = buildManyToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


d:\program\python\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
d:\program\python\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(30, 10))`
  """


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5684 samples, validate on 631 samples
Epoch 1/1000
5684/5684 [==============================] - 1s 168us/step - loss: 0.0050 - val_loss: 0.0013
Epoch 2/1000
5684/5684 [==============================] - 0s 57us/step - loss: 8.8673e-04 - val_loss: 4.7710e-04
Epoch 3/1000
5684/5684 [==============================] - 0s 54us/step - loss: 4.0300e-04 - val_loss: 2.5999e-04
Epoch 4/1000
5684/5684 [==============================] - 0s 55us/step - loss: 2.3340e-04 - val_loss: 1.6843e-04
Epoch 5/1000
5684/5684 [==

5684/5684 [==============================] - 0s 77us/step - loss: 2.9613e-05 - val_loss: 3.4229e-05
Epoch 68/1000
5684/5684 [==============================] - 0s 78us/step - loss: 2.8661e-05 - val_loss: 4.0018e-05
Epoch 69/1000
5684/5684 [==============================] - 0s 77us/step - loss: 2.9721e-05 - val_loss: 3.3491e-05
Epoch 70/1000
5684/5684 [==============================] - 0s 78us/step - loss: 2.8687e-05 - val_loss: 3.4081e-05
Epoch 71/1000
5684/5684 [==============================] - 0s 76us/step - loss: 2.8242e-05 - val_loss: 3.2888e-05
Epoch 72/1000
5684/5684 [==============================] - 0s 77us/step - loss: 2.7980e-05 - val_loss: 3.2435e-05
Epoch 73/1000
5684/5684 [==============================] - 0s 77us/step - loss: 2.8782e-05 - val_loss: 3.3198e-05
Epoch 74/1000
5684/5684 [==============================] - 0s 78us/step - loss: 2.8100e-05 - val_loss: 3.1539e-05
Epoch 75/1000
5684/5684 [==============================] - 0s 79us/step - loss: 2.8196e-05 - val_loss:

In [50]:
## build 1-n model

def buildOneToManyModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
  # output shape: (5, 1)
  model.add(Dense(1))
  model.add(RepeatVector(5))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [51]:
## run 1-n model 

train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildOneToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


d:\program\python\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
d:\program\python\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(1, 10))`
  """


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 1)              0         
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5706 samples, validate on 634 samples
Epoch 1/1000
5706/5706 [==============================] - 1s 132us/step - loss: 0.0102 - val_loss: 0.0028
Epoch 2/1000
5706/5706 [==============================] - 0s 11us/step - loss: 9.3778e-04 - val_loss: 2.9156e-04
Epoch 3/1000
5706/5706 [==============================] - 0s 10us/step - loss: 2.7569e-04 - val_loss: 2.4244e-04
Epoch 4

5706/5706 [==============================] - 0s 12us/step - loss: 6.0635e-05 - val_loss: 5.7510e-05
Epoch 67/1000
5706/5706 [==============================] - 0s 12us/step - loss: 6.0637e-05 - val_loss: 5.7646e-05
Epoch 68/1000
5706/5706 [==============================] - 0s 12us/step - loss: 6.0439e-05 - val_loss: 5.8870e-05
Epoch 69/1000
5706/5706 [==============================] - 0s 12us/step - loss: 5.9618e-05 - val_loss: 5.7251e-05
Epoch 70/1000
5706/5706 [==============================] - 0s 12us/step - loss: 6.0230e-05 - val_loss: 5.8343e-05
Epoch 71/1000
5706/5706 [==============================] - 0s 12us/step - loss: 6.0246e-05 - val_loss: 5.7281e-05
Epoch 72/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.9448e-05 - val_loss: 5.8081e-05
Epoch 73/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.9592e-05 - val_loss: 5.6669e-05
Epoch 74/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.9345e-05 - val_loss:

In [65]:
## build n-n model

def buildManyToManyModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))
  # output shape: (5, 1)
  model.add(TimeDistributed(Dense(1)))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [68]:
## run n-n model

train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 5, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildManyToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

pred = model.predict(X_train)

d:\program\python\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
d:\program\python\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(5, 10))`
  """


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 5, 10)             840       
_________________________________________________________________
time_distributed_4 (TimeDist (None, 5, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5703 samples, validate on 633 samples
Epoch 1/1000
5703/5703 [==============================] - 0s 83us/step - loss: 0.0422 - val_loss: 0.0181
Epoch 2/1000
5703/5703 [==============================] - 0s 35us/step - loss: 0.0111 - val_loss: 0.0062
Epoch 3/1000
5703/5703 [==============================] - 0s 28us/step - loss: 0.0060 - val_loss: 0.0048
Epoch 4/1000
5703/5703 [==============================] - 0s 36us/step - loss: 0.0046 - val_loss: 0.0037
Epoch 5/1000
5703/5703 [===========================

5703/5703 [==============================] - 0s 34us/step - loss: 9.8812e-05 - val_loss: 1.0876e-04
Epoch 68/1000
5703/5703 [==============================] - 0s 30us/step - loss: 9.8778e-05 - val_loss: 1.0209e-04
Epoch 69/1000
5703/5703 [==============================] - 0s 28us/step - loss: 9.6744e-05 - val_loss: 1.0359e-04
Epoch 70/1000
5703/5703 [==============================] - 0s 35us/step - loss: 9.7634e-05 - val_loss: 1.0278e-04
Epoch 71/1000
5703/5703 [==============================] - 0s 33us/step - loss: 9.7015e-05 - val_loss: 1.0213e-04
Epoch 72/1000
5703/5703 [==============================] - 0s 34us/step - loss: 9.7440e-05 - val_loss: 1.0278e-04
Epoch 73/1000
5703/5703 [==============================] - 0s 30us/step - loss: 9.6486e-05 - val_loss: 1.0345e-04
Epoch 74/1000
5703/5703 [==============================] - 0s 34us/step - loss: 9.6421e-05 - val_loss: 1.0045e-04
Epoch 75/1000
5703/5703 [==============================] - 0s 32us/step - loss: 9.5885e-05 - val_loss:

5703/5703 [==============================] - 0s 26us/step - loss: 8.9187e-05 - val_loss: 9.5304e-05
Epoch 139/1000
5703/5703 [==============================] - 0s 29us/step - loss: 8.9695e-05 - val_loss: 9.4911e-05
Epoch 140/1000
5703/5703 [==============================] - 0s 30us/step - loss: 9.0247e-05 - val_loss: 9.5618e-05
Epoch 141/1000
5703/5703 [==============================] - 0s 29us/step - loss: 8.8917e-05 - val_loss: 9.5309e-05
Epoch 142/1000
5703/5703 [==============================] - 0s 27us/step - loss: 8.9486e-05 - val_loss: 9.9071e-05
Epoch 143/1000
5703/5703 [==============================] - 0s 29us/step - loss: 8.8627e-05 - val_loss: 9.6157e-05
Epoch 144/1000
5703/5703 [==============================] - 0s 29us/step - loss: 9.0532e-05 - val_loss: 9.8265e-05
Epoch 145/1000
5703/5703 [==============================] - 0s 29us/step - loss: 8.9891e-05 - val_loss: 9.4332e-05
Epoch 146/1000
5703/5703 [==============================] - 0s 29us/step - loss: 8.9280e-05 - v

## Reference
[1] 李弘毅 — 機器學習 RNN
   http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2017/Lecture/RNN.pdf
   https://www.youtube.com/watch?v=xCGidAeyS4M

[2] Keras關於LSTM的units參數，還是不理解?
   https://www.zhihu.com/question/64470274

[3] Many to one and many to many LSTM examples in Keras
   https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras

[4] Yahoo — SPDR S&P 500 ETF (SPY)
   https://finance.yahoo.com/quote/SPY/history?period1=728236800&period2=1523462400&interval=1d&filter=history&frequency=1d      

[5] Wiki — 長短期記憶
   https://zh.wikipedia.org/wiki/%E9%95%B7%E7%9F%AD%E6%9C%9F%E8%A8%98%E6%86%B6